<a href="https://colab.research.google.com/github/Yaswanthyarra/MicroexpressionClassificationUsingCNNandGeneticAlgorithm/blob/main/BasicCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd '/content/drive/My Drive/CASMEII'

/content/drive/My Drive/CASMEII


In [3]:
dataset_dir = '/content/drive/My Drive/CASMEII/Cropped-updated/Cropped'

In [4]:
import numpy as np
import pandas as pd
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

# define constants
IMAGE_SIZE = (64, 64)
BATCH_SIZE = 32
NUM_EPOCHS = 60

# load the onset and apex frame indices from the Excel sheet
coding_file = pd.read_excel('/content/drive/My Drive/CASMEII/CASME2-coding-20140508.xlsx', sheet_name='Sheet1')

class_labels = coding_file['Estimated Emotion'].values

# define a function to extract the onset and apex frames for each expression
def extract_frames(sequence_path,sequence, subject_num):
    expression_images = []

    #To obtain the onset and apex frame numbers for the given subject and the corresponding sequence from the xlsheet
    subject_data = coding_file.loc[coding_file['Subject'] == subject_num]
    sequence_data = subject_data.loc[subject_data['Filename'] == sequence]
    onset_frame = sequence_data['OnsetFrame'].iloc[0]
    apex_frame = sequence_data['ApexFrame'].iloc[0]
                                                                                                                                                                 
    #To obtain the path to frames corresponding to the apex and onset frame numbers from the sequence of frames for the subject
    onset_frame_path = os.path.join(sequence_path,'reg_img' + str(onset_frame) + '.jpg')
    apex_frame_path = os.path.join(sequence_path, 'reg_img' + str(apex_frame) + '.jpg')
    print(subject_num,apex_frame_path)
    print(subject_num,onset_frame_path)
   
    #To obtain the apex and apex frames from path
    onset_frame = cv2.imread(onset_frame_path)
    apex_frame = cv2.imread(apex_frame_path)
    onset_frame = cv2.cvtColor(onset_frame, cv2.COLOR_BGR2GRAY)
    apex_frame = cv2.cvtColor(apex_frame, cv2.COLOR_BGR2GRAY)
    
    #To apply histogram equalisation to the frames to to enhance the contrast of an image by redistributing the pixel intensities. 
    onset_frame = cv2.equalizeHist(onset_frame)
    apex_frame = cv2.equalizeHist(apex_frame)

    #Resizing extracted frames
    onset_frame = cv2.resize(onset_frame, IMAGE_SIZE)
    apex_frame = cv2.resize(apex_frame, IMAGE_SIZE)

    #Stacking the apex and onset frames together and and appending to the expressin images list
    expression_images.append(np.stack([onset_frame, apex_frame], axis=-1))
    
    return np.array(expression_images)

# create a data generator for the image data
data_generator = ImageDataGenerator(rescale=1./255)

#Setting up several image augmentation parameters that can be used to generate augmented images 
exp_frame_augmentation = {
    'rotation_range': 10,
    'width_shift_range': 0.1,
    'height_shift_range': 0.1,
    'shear_range': 0.1,
    'zoom_range': 0.1,
    'horizontal_flip': True,
    'brightness_range': [0.5, 1.5],
}


# load the expression folders and extract the onset and apex frames for each subject

#converting the categorical class labels of a classification problem into numerical form
label_dict = {'happiness': 0, 'disgust': 1, 'repression': 2, 'surprise': 3, 'fear': 4, 'others': 6,'sadness':5}

expression_images = []
expression_labels = []

#Looping through all the subjects in the dataset
for subject in os.listdir(dataset_dir):

    subject_num = int(subject[3:])
    subject_dir = os.path.join(dataset_dir, subject)

    #Looping through each sequence for the subject
    for sequence in os.listdir(subject_dir):
        
        sequence_path = os.path.join(subject_dir, sequence)

        #Extracting the onset and apex frames by invoking the extract_frames method
        expression_frames = extract_frames(sequence_path, sequence, subject_num)

        #Obtaining the expression label for the sequence from the xlsheet and appending it to expression_label 
        subject_data = coding_file.loc[coding_file['Subject'] == subject_num]
        sequence_data = subject_data.loc[subject_data['Filename'] == sequence]
        exp_label_category = sequence_data['Estimated Emotion']
        expression_label = label_dict[exp_label_category.values[0]]

        #Adding the extracted frames and the correspoing class label to expression_images and expression_labels
        expression_images.append(expression_frames)
        expression_labels.append(expression_label)

        # Generating augmented images for the fear class
        if expression_label == 4:  # if fear class
            
            for i in range(30):
              print(i,subject_num)

              #Generating the augmented apex and onset frames
              apex_augmented=data_generator.apply_transform(expression_frames[:, :, :, 0], exp_frame_augmentation)
              onset_augmented=data_generator.apply_transform(expression_frames[:, :, :, 1], exp_frame_augmentation)
              
              #Appending the augmented images to expression_images and expression_labels lists
              expression_images.append(np.stack([apex_augmented, onset_augmented], axis=-1))
              expression_labels.append(expression_label)
        
        # Generating augmented images for the sadness class
        if expression_label == 5:  # if sadness class
           
            for i in range(9):
              print(i,subject_num)
              
              #Generating the augmented apex and onset frames
              apex_augmented=data_generator.apply_transform(expression_frames[:, :, :, 0], exp_frame_augmentation)
              onset_augmented=data_generator.apply_transform(expression_frames[:, :, :, 1], exp_frame_augmentation)
              

              #Appending the augmented images to expression_images and expression_labels lists
              expression_images.append(np.stack([apex_augmented, onset_augmented], axis=-1))
              expression_labels.append(expression_label)  

        # generate augmented images for the happiness class
        if expression_label == 0:  # if happiness class
        
            for i in range(1):
              print(i,subject_num)

              #Generating the augmented apex and onset frames
              apex_augmented=data_generator.apply_transform(expression_frames[:, :, :, 0], exp_frame_augmentation)
              onset_augmented=data_generator.apply_transform(expression_frames[:, :, :, 1], exp_frame_augmentation)
              
              #Appending the augmented images to expression_images and expression_labels lists
              expression_images.append(np.stack([apex_augmented, onset_augmented], axis=-1))
              expression_labels.append(expression_label)  

        # Generate augmented images for the repression class
        if expression_label == 2:  # if repression class
          
            for i in range(1):
              print(i,subject_num)
              apex_augmented=data_generator.apply_transform(expression_frames[:, :, :, 0], exp_frame_augmentation)
              onset_augmented=data_generator.apply_transform(expression_frames[:, :, :, 1], exp_frame_augmentation)

              #Appending the augmented images to expression_images and expression_labels lists
              expression_images.append(np.stack([apex_augmented, onset_augmented], axis=-1))
              expression_labels.append(expression_label)  
        
        # Generate augmented images for the surprise class
        if expression_label == 3:  # if surprise class
            
            
            for i in range(1):
              print(i,subject_num)

              #Generating the augmented apex and onset frames
              apex_augmented=data_generator.apply_transform(expression_frames[:, :, :, 0], exp_frame_augmentation)
              onset_augmented=data_generator.apply_transform(expression_frames[:, :, :, 1], exp_frame_augmentation)

              #Appending the augmented images to expression_images and expression_labels lists
              expression_images.append(np.stack([apex_augmented, onset_augmented], axis=-1))
              expression_labels.append(expression_label)  


        

        
# convert the data to numpy arrays 
expression_images = np.concatenate(expression_images, axis=0)
expression_labels = np.array(expression_labels)

#To obtain the number of class labels
num_classes = len(np.unique(expression_labels))

#Stratified Spliting of data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(expression_images, expression_labels, test_size=0.2,stratify=expression_labels,  random_state=42)



25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP09_02/reg_img101.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP09_02/reg_img76.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP10_01/reg_img129.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP10_01/reg_img101.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP03_01/reg_img81.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP03_01/reg_img56.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP03_02/reg_img101.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP03_02/reg_img51.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP10_10/reg_img91.jpg
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP10_10/reg_img61.jpg
0 25
25 /content/drive/My Drive/CASMEII/Cropped-updated/Cropped/sub25/EP18_04f/reg_img116.jpg
25 /content/drive/My Drive/CASME

In [10]:
from keras.optimizers import Adam
# define the model architecture
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 2)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.33),
    Dense(num_classes, activation='softmax')
])
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=30, batch_size=64, validation_data=(test_images, test_labels))

# evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print('Test accuracy:', test_acc)

Epoch 1/30
6/6 [==============================] - 5s 493ms/step - loss: 39.0851 - accuracy: 0.1680 - val_loss: 8.8983 - val_accuracy: 0.1720
Epoch 2/30
6/6 [==============================] - 3s 461ms/step - loss: 8.9546 - accuracy: 0.1734 - val_loss: 2.8026 - val_accuracy: 0.2258
Epoch 3/30
6/6 [==============================] - 3s 480ms/step - loss: 2.8494 - accuracy: 0.2168 - val_loss: 1.8211 - val_accuracy: 0.2366
Epoch 4/30
6/6 [==============================] - 4s 737ms/step - loss: 1.8721 - accuracy: 0.2222 - val_loss: 1.7846 - val_accuracy: 0.3011
Epoch 5/30
6/6 [==============================] - 6s 945ms/step - loss: 1.8019 - accuracy: 0.2060 - val_loss: 1.7155 - val_accuracy: 0.2581
Epoch 6/30
6/6 [==============================] - 3s 481ms/step - loss: 1.7593 - accuracy: 0.2385 - val_loss: 1.7038 - val_accuracy: 0.3226
Epoch 7/30
6/6 [==============================] - 3s 455ms/step - loss: 1.6969 - accuracy: 0.3117 - val_loss: 1.5934 - val_accuracy: 0.4409
Epoch 8/30
6/6 [===

In [5]:
#Using Optimised hyperparameters
from keras.optimizers import Adam
# define the model architecture
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 2)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.29999999999999993),
    Dense(num_classes, activation='softmax')
])
optimizer = Adam(learning_rate=0.0006651508397586441)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=52, batch_size=17, validation_data=(test_images, test_labels))

# evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print('Test accuracy:', test_acc)

Epoch 1/52
22/22 [==============================] - 6s 199ms/step - loss: 19.7892 - accuracy: 0.1436 - val_loss: 1.9158 - val_accuracy: 0.1935
Epoch 2/52
22/22 [==============================] - 3s 135ms/step - loss: 1.8468 - accuracy: 0.2683 - val_loss: 1.3993 - val_accuracy: 0.5161
Epoch 3/52
22/22 [==============================] - 3s 131ms/step - loss: 1.4150 - accuracy: 0.4797 - val_loss: 1.1218 - val_accuracy: 0.5269
Epoch 4/52
22/22 [==============================] - 4s 165ms/step - loss: 1.1851 - accuracy: 0.5556 - val_loss: 1.1651 - val_accuracy: 0.5591
Epoch 5/52
22/22 [==============================] - 4s 197ms/step - loss: 1.1021 - accuracy: 0.5989 - val_loss: 1.0270 - val_accuracy: 0.6667
Epoch 6/52
22/22 [==============================] - 3s 130ms/step - loss: 0.8913 - accuracy: 0.6911 - val_loss: 0.6921 - val_accuracy: 0.7742
Epoch 7/52
22/22 [==============================] - 3s 131ms/step - loss: 0.7366 - accuracy: 0.7154 - val_loss: 0.6367 - val_accuracy: 0.7527
Epoch

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

path = '/content/drive/My Drive/CASMEII/microexp_cnn_model.hdf5'

# save model to path
tf.keras.models.save_model(model, path)